In [22]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("../")

import gurobipy
from json import dumps, loads
from time import time

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as skLogisticRegression
from sklearn.metrics import (classification_report, f1_score, precision_score, recall_score)
from tqdm import tnrange, trange
import tensorflow as tf

from mlsql.influence import InfluenceRanker
from mlsql.fixer import AutoFixer
from mlsql.manager import ModelManagerLM
from mlsql.manager_test import ModelManagerTest

from models.simple_cnn import SimpleCNN
from models.logreg import LogReg
from models.linear_comb import LinearComb
from models.linear_comb_test import LinearCombTest
from processors.mnistbinary_7 import MnistBinaryProcessor


import logging
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)

import time
import altair as alt
alt.data_transformers.disable_max_rows()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


DataTransformerRegistry.enable('default')

In [23]:
# @tf.function
def rank_fix(ranker, fixer, n):
    rank = ranker.predict()
    fixer.fix(rank, n)
    return rank

@tf.function
def rankit(ranker):
    rank = ranker.predict()
    return rank


@tf.function
def fixit(fixer, rank, n):
    fixer.fix(rank, n)


# @tf.function
def train(manager):
    manager.fit()

In [24]:
proc = MnistBinaryProcessor()

In [25]:
model = LogReg(1)
manager0 = ModelManagerLM(proc.x_train, proc.y_corr, model, 256)
start = time.time()
manager0.fit(print_value=True, tol=1e-5, lr=0.1, max_iter=2000)
print(time.time() - start)
manager0.report(proc.x_train, proc.y_corr, proc.x_test, proc.y_test)
manager0.report(proc.x_train, proc.y_corr, proc.x_query, proc.y_query)

SGD loss: tf.Tensor(0.367625, shape=(), dtype=float32)
SGD steps: 356
40.59802484512329
Model name: LogReg
On Training
               precision    recall  f1-score   support

         0.0       0.85      1.00      0.92     45776
         1.0       0.22      0.00      0.00      8224

    accuracy                           0.85     54000
   macro avg       0.54      0.50      0.46     54000
weighted avg       0.75      0.85      0.78     54000

On Testing
               precision    recall  f1-score   support

         0.0       0.49      1.00      0.66      4926
         1.0       1.00      0.00      0.01      5074

    accuracy                           0.50     10000
   macro avg       0.75      0.50      0.34     10000
weighted avg       0.75      0.50      0.33     10000

Model name: LogReg
On Training
               precision    recall  f1-score   support

         0.0       0.85      1.00      0.92     45776
         1.0       0.22      0.00      0.00      8224

    accuracy      

In [26]:
K = 19189
corrsel = proc.corrsel
print(len(list(np.where(corrsel)[0])))

19189


In [41]:
from tqdm.notebook import tnrange, trange
manager = ModelManagerLM(proc.x_train, proc.y_corr, LogReg(1), 256)
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
from mlsql.loss_ranker import LossRanker
ranker = LossRanker(manager=manager)
# ranker = InfluenceRanker(manager=manager, on=proc.complain)
fixer = AutoFixer(manager, corrsel, K)

AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_rain = 0
# AQ = proc.complain(manager).AQ
# f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
f1 = f1_score(proc.y_query.numpy(), manager.model.predict(proc.x_query).numpy(), average='weighted')
# AQs.append(float(AQ))
weighted_f1.append(f1)

step_size = 10
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
    rank = rank_fix(ranker, fixer, nfix)
    middle = time.time()
    manager.fit(max_iter=1000, print_value=True, lr=0.1, tol=1e-5)
    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += end - middle

#     AQ = proc.complain(manager).AQ
#     f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
    f1 = f1_score(proc.y_query.numpy(), manager.model.predict(proc.x_query).numpy(), average='weighted')
#     AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_rain)
print("Model_time:", model_time_rain)
# AC = proc.complain(manager).AC

df_rain = pd.DataFrame({
#     "Complain": np.array(AQs) - AC,
    "F1": np.array(weighted_f1),
    "K": list(range(0, K, step_size)) + [K],
    "Method": np.repeat("Rain", len(weighted_f1)),
})
alt.Chart(pd.concat([df_rain])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain.shape[0], grid=False)),
    alt.Y("F1:Q"),
    color = "Method"
)

  0%|          | 0/1919 [00:00<?, ?it/s]

SGD loss: tf.Tensor(0.36678332, shape=(), dtype=float32)
SGD steps: 9
SGD loss: tf.Tensor(0.36600056, shape=(), dtype=float32)
SGD steps: 9
SGD loss: tf.Tensor(0.365266, shape=(), dtype=float32)
SGD steps: 8
SGD loss: tf.Tensor(0.3645624, shape=(), dtype=float32)
SGD steps: 8
SGD loss: tf.Tensor(0.3638785, shape=(), dtype=float32)
SGD steps: 8
SGD loss: tf.Tensor(0.36321157, shape=(), dtype=float32)
SGD steps: 8
SGD loss: tf.Tensor(0.36256206, shape=(), dtype=float32)
SGD steps: 7
SGD loss: tf.Tensor(0.36190385, shape=(), dtype=float32)
SGD steps: 9
SGD loss: tf.Tensor(0.36130732, shape=(), dtype=float32)
SGD steps: 4
SGD loss: tf.Tensor(0.36068422, shape=(), dtype=float32)
SGD steps: 7
SGD loss: tf.Tensor(0.36009023, shape=(), dtype=float32)
SGD steps: 5
SGD loss: tf.Tensor(0.3594628, shape=(), dtype=float32)
SGD steps: 9
SGD loss: tf.Tensor(0.3588685, shape=(), dtype=float32)
SGD steps: 6
SGD loss: tf.Tensor(0.35828772, shape=(), dtype=float32)
SGD steps: 5
SGD loss: tf.Tensor(0.3577

KeyboardInterrupt: 

In [37]:
fixer.recall_k()

array([0.        , 0.        , 0.        , ..., 0.50586273, 0.50591485,
       0.50596696])

In [38]:
manager0.report(proc.x_train, proc.y_corr, proc.x_test, proc.y_test)


Model name: LogReg
On Training
               precision    recall  f1-score   support

         0.0       0.85      1.00      0.92     45776
         1.0       0.22      0.00      0.00      8224

    accuracy                           0.85     54000
   macro avg       0.54      0.50      0.46     54000
weighted avg       0.75      0.85      0.78     54000

On Testing
               precision    recall  f1-score   support

         0.0       0.49      1.00      0.66      4926
         1.0       1.00      0.00      0.01      5074

    accuracy                           0.50     10000
   macro avg       0.75      0.50      0.34     10000
weighted avg       0.75      0.50      0.33     10000



In [40]:
np.concatenate((fixer.recall_k()[0::500], fixer.recall_k()[-1:]))

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01443535, 0.04049195, 0.06628798,
       0.09218823, 0.11782792, 0.14305071, 0.1683256 , 0.19323571,
       0.21804159, 0.24201365, 0.26614206, 0.28990568, 0.31215801,
       0.33540049, 0.35822607, 0.38094742, 0.40179269, 0.42169993,
       0.44150294, 0.4610975 , 0.48079629, 0.49934859, 0.50596696])

In [ ]:
# LinearCombTest
model = LinearCombTest(1)
manager_test0 = ModelManagerTest(proc.x_a_train, proc.x_b_train, proc.y_corr, model, 256)
start = time.time()
manager_test0.fit(print_value=True, tol=1e-9, lr=0.1, max_iter=50000)
print(time.time() - start)
manager_test0.report(proc.x_a_train, proc.x_b_train, proc.y_corr, proc.x_a_test, proc.x_b_test, proc.y_test)
manager_test0.report(proc.x_a_train, proc.x_b_train, proc.y_corr, proc.x_a_query, proc.x_b_query, proc.y_query)

In [ ]:
manager_test = ModelManagerTest(proc.x_a_train, proc.x_b_train, proc.y_corr, LinearCombTest(1), 256)
manager_test.model.set_weights(manager_test0.model.get_weights())
manager_test.delta = tf.Variable(manager_test0.delta.value(), name="delta")
ranker = InfluenceRanker(manager=manager_test, on=proc.test_complain)
fixer = AutoFixer(manager_test, proc.corrsel, K)

AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_rain = 0
AQ = proc.test_complain(manager_test).AQ
# f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
AQs.append(float(AQ))
weighted_f1.append(f1)

step_size = 19189
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
    rank = rank_fix(ranker, fixer, nfix)
    middle = time.time()
    manager_test.fit(max_iter=5000, tol=1e-8, lr=0.1, print_value=True)
    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += end - middle

    AQ = proc.test_complain(manager_test).AQ
#     f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
    f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
    AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_rain)
print("Model_time:", model_time_rain)
AC = proc.test_complain(manager_test).AC

df_rain_test = pd.DataFrame({
    "Complain": np.array(AQs) - AC,
    "F1": np.array(weighted_f1),
    "K": list(range(0, K, step_size)) + [K],
    "Method": np.repeat("Rain", len(AQs)),
})
alt.Chart(pd.concat([df_rain_test])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain_test.shape[0], grid=False)),
    alt.Y("Complain:Q"),
    color = "Method"
)

In [ ]:
np.concatenate((fixer.recall_k()[0::1000], fixer.recall_k()[-1:]))